In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform


In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)

In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [4]:
data = data[data.index.isin(data_target.index)]

In [6]:
data_target

,category_tnk_1,category_tnk_2,category_tnk_3,category_tnk_4,category_tnk_5,category_tnk_6,category_tnk_7,category_tnk_8,category_tnk_9,category_tnk_10,...,category_stromal_62,category_stromal_63,category_stromal_64,category_stromal_65,category_stromal_66,category_stromal_68,category_stromal_73,category_stromal_78,category_stromal_80,category_stromal_85
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-6142,1,1,0,0,0,0,0,0,-1,0,...,1,1,1,1,1,-1,0,0,-1,-1
TCGA-5M-AAT4,-1,-1,-1,0,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-2676,1,1,1,0,1,1,1,1,1,1,...,0,-1,0,-1,0,1,0,1,1,1
TCGA-A6-6652,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,1,1,1,-1,1,1,1,1,-1,1,...,0,0,-1,1,1,-1,1,1,-1,-1
TCGA-RU-A8FL,-1,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-G4-6303,-1,0,-1,-1,0,0,0,-1,-1,1,...,1,1,1,0,1,-1,0,0,-1,-1


In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.7323592099410596, 'gamma': 0.03177917514301182, 'learning_rate': 0.10329469651469865, 'max_depth': 10, 'min_child_weight': 5, 'n_estimators': 198, 'subsample': 0.8365191150830908}
Cross-validation F1 scores on training set: [0.50189167 0.41718132 0.49907191 0.39585521 0.52024264]
Average cross-validation F1 score on training set: 0.466848549262388
Classification Report for category_b_12 with 150 features (Train Set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       120

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Classification Report for category_b_12 with 150 features (Test Set)

In [5]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)

# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Step 1: Initialize XGBoost Classifier and RFE
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
rfe = RFE(estimator=xgb_clf, n_features_to_select=100)  # Choose top 100 features

# Fit RFE to select top 100 features
X_train_rfe = rfe.fit_transform(X_train, Y_train)
X_test_rfe = rfe.transform(X_test)

# Step 2: Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Perform Randomized Search with cross-validation on the selected features
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_rfe, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train_rfe, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Step 3: Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train_rfe, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train_rfe)
print("Classification Report for category_b_12 with 100 selected features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test_rfe)
print("Classification Report for category_b_12 with 100 selected features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9915571433100037, 'gamma': 0.24337107647972755, 'learning_rate': 0.2818296363155662, 'max_depth': 13, 'min_child_weight': 2, 'n_estimators': 376, 'subsample': 0.8010548372420768}
Cross-validation F1 scores on training set: [0.65268784 0.42676768 0.57422478 0.54278813 0.48839565]
Average cross-validation F1 score on training set: 0.5369728161712064
Classification Report for category_b_12 with 100 selected features (Train Set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       120

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Classification Report for category_b_12 with 100 selected f

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)

# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Step 1: Train an initial XGBoost model to get feature importances
initial_xgb = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
initial_xgb.fit(X_train, Y_train)

# Get feature importances and select the top 100 features
feature_importances = initial_xgb.feature_importances_
top_100_indices = np.argsort(feature_importances)[-20:]  # Indices of the top 100 features

# Subset training and testing sets to only include the top 100 features
X_train_top100 = X_train[:, top_100_indices]
X_test_top100 = X_test[:, top_100_indices]

# Step 2: Set up the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Step 3: Perform Randomized Search with cross-validation on the top 100 features
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_top100, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train_top100, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Step 4: Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train_top100, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train_top100)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test_top100)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.8887995089067299, 'gamma': 0.4692763545078751, 'learning_rate': 0.010233629752304298, 'max_depth': 14, 'min_child_weight': 5, 'n_estimators': 260, 'subsample': 0.7216968971838151}
Cross-validation F1 scores on training set: [0.36766222 0.4175359  0.46       0.44242424 0.48756004]
Average cross-validation F1 score on training set: 0.4350364813458891
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       120
           1       0.96      0.97      0.96       119
           2       0.98      0.96      0.97       120

    accuracy                           0.97       359
   macro avg       0.97      0.97      0.97       359
weighted avg       0.97      0.97      0.97       359

Classification Report for category_b_12 with 100 features (Test Set

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming data (X) and data_target (Y) are your feature and target datasets
X = data.values

# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

# Convert the target labels to sequential values using LabelEncoder
label_encoder = LabelEncoder()
Y_category_b_12_encoded = label_encoder.fit_transform(Y_category_b_12)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12_encoded, test_size=0.2, random_state=42)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
train_report = classification_report(Y_train, Y_train_pred)
            
# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

# Predict on the test set using the best model
Y_pred = best_xgb.predict(X_test)

# Decode the predictions and true labels back to the original labels
Y_pred_decoded = label_encoder.inverse_transform(Y_pred)
Y_test_decoded = label_encoder.inverse_transform(Y_test)

# Evaluate the model
print("Classification Report for category_b_12 using XGBoost with tuned hyperparameters:")
print(classification_report(Y_test_decoded, Y_pred_decoded))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'colsample_bytree': 0.9812287388095813, 'gamma': 0.16537515233525685, 'learning_rate': 0.175829490050647, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 215, 'subsample': 0.7177795568278342}
Classification Report for category_b_12 using XGBoost with tuned hyperparameters:
              precision    recall  f1-score   support

          -1       0.62      0.64      0.63        33
           0       0.45      0.52      0.48        27
           1       0.56      0.47      0.51        30

    accuracy                           0.54        90
   macro avg       0.54      0.54      0.54        90
weighted avg       0.55      0.54      0.54        90

